In [1]:
import shutil
import os
import pandas as pd
import numpy as np
import wget

pd.set_option('display.max_columns', 100)

# Data retrieval

In [2]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# remove existing files in the folder
files_to_rm = ['time_series_covid19_deaths_global.csv',
               'time_series_covid19_recovered_global.csv',
               'time_series_covid19_confirmed_global.csv',]

# shutil.copy('covid.csv', './old_data')
try:
    for f in files_to_rm:
        os.remove(f)
except:
    pass
        
for url in urls:
    filename = wget.download(url)


'''
# make sure updated dataset is available only and not losing the locally saved
try:
    for url in urls:
        filename = wget.download(url)
except:
    for f in files_to_rm:
        os.remove(f)
    for url in urls:
        filename = wget.download(url)
        '''

    
deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv')
confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')

100% [............................................................................] 188384 / 188384

In [3]:
root_dir = 'C:\\Users\\dongt\\Documents\\data science\\projects\\capstone 2'
source_dir = root_dir + '\\notebooks'
target_dir = root_dir + '\\data\\raw'

In [4]:
files_to_move = ['time_series_covid19_deaths_global.csv',
               'time_series_covid19_recovered_global.csv',
               'time_series_covid19_confirmed_global.csv',]

In [5]:
# move files to /data/raw
def move_files(files_to_move, source_dir, target_dir):
    for file in files_to_move:
        shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))

In [6]:
move_files(files_to_move, source_dir, target_dir)

In [7]:
value_vars = deaths.columns[4:]

deaths = deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                     var_name='Date', value_vars=value_vars, value_name='Deaths')

recovered = recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                           var_name='Date', value_vars=value_vars, value_name='Recovered')

confirmed = confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                           var_name='Date', value_vars=value_vars, value_name='Confirmed')

In [8]:
recovered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51359 entries, 0 to 51358
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  13601 non-null  object 
 1   Country/Region  51359 non-null  object 
 2   Lat             51359 non-null  float64
 3   Long            51359 non-null  float64
 4   Date            51359 non-null  object 
 5   Recovered       51359 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 2.4+ MB


In [9]:
# get a list of states in Canada
confirmed[confirmed['Country/Region']=='Canada']
confirmed_canada = confirmed[confirmed['Country/Region']=='Canada']
can_states = list(confirmed_canada['Province/State'].unique()) # using confirmed data eleminate all cruises rows in recovered data

# number of Canadian states
n = len(confirmed_canada['Province/State'].unique())
recovered_ca = recovered[recovered['Country/Region']=='Canada']

In [10]:
recovered_ca

,Province/State,Country/Region,Lat,Long,Date,Recovered
36,NaN,Canada,56.1304,-106.3468,1/22/20,0
289,NaN,Canada,56.1304,-106.3468,1/23/20,0
542,NaN,Canada,56.1304,-106.3468,1/24/20,0
795,NaN,Canada,56.1304,-106.3468,1/25/20,0
1048,NaN,Canada,56.1304,-106.3468,1/26/20,0
...,...,...,...,...,...,...
50130,NaN,Canada,56.1304,-106.3468,8/7/20,105128
50383,NaN,Canada,56.1304,-106.3468,8/8/20,105255
50636,NaN,Canada,56.1304,-106.3468,8/9/20,105420
50889,NaN,Canada,56.1304,-106.3468,8/10/20,108050


- There is no information for State in Canada rows. In order to merge the canada Recovered to the dataset, where each state of canada has the data for deaths and confirmed, it is better to convert canada Recovered having the the recovered data for each state. For convenience reason, the confirmed for each canadian state will be the same, total confirmed devided by the  number of states.

In [11]:
recovered_canada = recovered_ca.copy()
for state in can_states:
    recovered_canada[state] = recovered_ca.loc[:, 'Recovered'] / n
recovered_canada = recovered_canada.drop(['Recovered'], axis=1)

In [12]:
recovered_canada

,Province/State,Country/Region,Lat,Long,Date,Alberta,British Columbia,Grand Princess,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan,Diamond Princess,Northwest Territories,Yukon
36,NaN,Canada,56.1304,-106.3468,1/22/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
289,NaN,Canada,56.1304,-106.3468,1/23/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
542,NaN,Canada,56.1304,-106.3468,1/24/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
795,NaN,Canada,56.1304,-106.3468,1/25/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1048,NaN,Canada,56.1304,-106.3468,1/26/20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50130,NaN,Canada,56.1304,-106.3468,8/7/20,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857,7509.142857
50383,NaN,Canada,56.1304,-106.3468,8/8/20,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286,7518.214286
50636,NaN,Canada,56.1304,-106.3468,8/9/20,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000,7530.000000
50889,NaN,Canada,56.1304,-106.3468,8/10/20,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143,7717.857143


In [13]:
can_states = recovered_canada.columns[5:]

In [14]:
recovered_canada = recovered_canada.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'],
        var_name='State', value_vars=can_states, value_name='Recovered')

In [15]:
recovered_canada

,Province/State,Country/Region,Lat,Long,Date,State,Recovered
0,NaN,Canada,56.1304,-106.3468,1/22/20,Alberta,0.000000
1,NaN,Canada,56.1304,-106.3468,1/23/20,Alberta,0.000000
2,NaN,Canada,56.1304,-106.3468,1/24/20,Alberta,0.000000
3,NaN,Canada,56.1304,-106.3468,1/25/20,Alberta,0.000000
4,NaN,Canada,56.1304,-106.3468,1/26/20,Alberta,0.000000
...,...,...,...,...,...,...,...
2837,NaN,Canada,56.1304,-106.3468,8/7/20,Yukon,7509.142857
2838,NaN,Canada,56.1304,-106.3468,8/8/20,Yukon,7518.214286
2839,NaN,Canada,56.1304,-106.3468,8/9/20,Yukon,7530.000000
2840,NaN,Canada,56.1304,-106.3468,8/10/20,Yukon,7717.857143


In [16]:
recovered_canada['Province/State'] = recovered_canada['State']

In [17]:
recovered_canada

,Province/State,Country/Region,Lat,Long,Date,State,Recovered
0,Alberta,Canada,56.1304,-106.3468,1/22/20,Alberta,0.000000
1,Alberta,Canada,56.1304,-106.3468,1/23/20,Alberta,0.000000
2,Alberta,Canada,56.1304,-106.3468,1/24/20,Alberta,0.000000
3,Alberta,Canada,56.1304,-106.3468,1/25/20,Alberta,0.000000
4,Alberta,Canada,56.1304,-106.3468,1/26/20,Alberta,0.000000
...,...,...,...,...,...,...,...
2837,Yukon,Canada,56.1304,-106.3468,8/7/20,Yukon,7509.142857
2838,Yukon,Canada,56.1304,-106.3468,8/8/20,Yukon,7518.214286
2839,Yukon,Canada,56.1304,-106.3468,8/9/20,Yukon,7530.000000
2840,Yukon,Canada,56.1304,-106.3468,8/10/20,Yukon,7717.857143


In [18]:
# drop column 'State'
recovered_canada = recovered_canada.drop(['State'], axis=1)

In [19]:
recovered_canada

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,Alberta,Canada,56.1304,-106.3468,1/22/20,0.000000
1,Alberta,Canada,56.1304,-106.3468,1/23/20,0.000000
2,Alberta,Canada,56.1304,-106.3468,1/24/20,0.000000
3,Alberta,Canada,56.1304,-106.3468,1/25/20,0.000000
4,Alberta,Canada,56.1304,-106.3468,1/26/20,0.000000
...,...,...,...,...,...,...
2837,Yukon,Canada,56.1304,-106.3468,8/7/20,7509.142857
2838,Yukon,Canada,56.1304,-106.3468,8/8/20,7518.214286
2839,Yukon,Canada,56.1304,-106.3468,8/9/20,7530.000000
2840,Yukon,Canada,56.1304,-106.3468,8/10/20,7717.857143


In [20]:
recovered_wo_canada = recovered[recovered['Country/Region']!='Canada']

In [21]:
recovered_wo_canada

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
51354,NaN,Sao Tome and Principe,0.186400,6.613100,8/11/20,804
51355,NaN,Yemen,15.552727,48.516388,8/11/20,915
51356,NaN,Comoros,-11.645500,43.333300,8/11/20,379
51357,NaN,Tajikistan,38.861000,71.276100,8/11/20,6653


In [22]:
# combine vertically
recovered = recovered_wo_canada.append(recovered_canada)

In [23]:
recovered

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0.000000
1,NaN,Albania,41.15330,20.168300,1/22/20,0.000000
2,NaN,Algeria,28.03390,1.659600,1/22/20,0.000000
3,NaN,Andorra,42.50630,1.521800,1/22/20,0.000000
4,NaN,Angola,-11.20270,17.873900,1/22/20,0.000000
...,...,...,...,...,...,...
2837,Yukon,Canada,56.13040,-106.346800,8/7/20,7509.142857
2838,Yukon,Canada,56.13040,-106.346800,8/8/20,7518.214286
2839,Yukon,Canada,56.13040,-106.346800,8/9/20,7530.000000
2840,Yukon,Canada,56.13040,-106.346800,8/10/20,7717.857143


In [24]:
print('number of rows in deaths:', len(deaths))
print('number of rows in recovered:', len(recovered))
print('number of rows in confirmed:', len(confirmed))

number of rows in deaths: 53998
number of rows in recovered: 53998
number of rows in confirmed: 53998


In [25]:
df = pd.merge(left=confirmed, right=deaths, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
df = pd.merge(left=df, right=recovered, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53998 entries, 0 to 53997
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  16443 non-null  object 
 1   Country/Region  53998 non-null  object 
 2   Lat             53998 non-null  float64
 3   Long            53998 non-null  float64
 4   Date            53998 non-null  object 
 5   Confirmed       53998 non-null  int64  
 6   Deaths          53998 non-null  int64  
 7   Recovered       50141 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 3.7+ MB


In [27]:
df['Recovered'] = df['Recovered'].fillna(0)
df['Recovered'] = df['Recovered'].astype('int') # change from float to integer
df.isnull().sum()

Province/State    37555
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [28]:
# remove rows that Province/State is Recovered
df = df[df['Province/State'].str.contains('Recovered')!=True]

# modify the name for South Korea
df['Country/Region'] = df['Country/Region'].replace('Korea, South', 'South Korea')

In [29]:
# update data for Hubei confirmed on 2/12/20
df.loc[(df['Date']=='2/12/20') & (df['Province/State']=='Hubei'), 'Confirmed']

5648    33366
Name: Confirmed, dtype: int64

In [30]:
info = ['Hubei', '2/12/20', 'Confirmed', 34874]
df.loc[(df['Province/State']==info[0]) & (df['Date']==info[1]), info[2]] = info[3]
df.loc[(df['Province/State']==info[0]) & (df['Date']==info[1]), info[2]]

5648    34874
Name: Confirmed, dtype: int64

In [31]:
df['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [32]:
df[df['Country/Region']=='US']

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
225,NaN,US,40.0,-100.0,1/22/20,1,0,0
491,NaN,US,40.0,-100.0,1/23/20,1,0,0
757,NaN,US,40.0,-100.0,1/24/20,2,0,0
1023,NaN,US,40.0,-100.0,1/25/20,2,0,0
1289,NaN,US,40.0,-100.0,1/26/20,5,0,0
...,...,...,...,...,...,...,...,...
52893,NaN,US,40.0,-100.0,8/7/20,4941755,161347,1623870
53159,NaN,US,40.0,-100.0,8/8/20,4997929,162423,1643118
53425,NaN,US,40.0,-100.0,8/9/20,5044864,162938,1656864
53691,NaN,US,40.0,-100.0,8/10/20,5094400,163463,1670755


In [33]:
# active cases
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

df.loc[df['Country/Region']=='US', 'Country/Region'] = 'United States'
df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')  

# filling missing values 
df[['Province/State']] = df[['Province/State']].fillna('')
df[['Confirmed', 'Deaths', 'Recovered', 'Active']] = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)


# fixing datatypes
df['Recovered'] = df['Recovered'].astype(int)

df.sample(6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
2989,Hunan,China,27.610400,111.708800,2/2/20,463,0,16,447
37733,,Venezuela,6.423800,-66.589700,6/11/20,2814,23,487,2304
31645,Falkland Islands (Malvinas),United Kingdom,-51.796300,-59.523600,5/19/20,13,0,13,0
4271,Western Australia,Australia,-31.950500,115.860500,2/7/20,0,0,0,0
4496,,West Bank and Gaza,31.952200,35.233200,2/7/20,0,0,0,0
43592,,Mozambique,-18.665695,35.529562,7/3/20,939,6,0,933


In [34]:
try:
    os.remove('../data/processed/covid.csv')
except:
    pass

df.to_csv('../data/processed/covid.csv', index=False)

In [76]:
covid = pd.read_csv('../data/processed/covid.csv')
covid[covid['Date']==max(covid['Date'])]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
53200,NaN,Afghanistan,33.939110,67.709953,8/9/20,37054,1312,25960,9782
53201,NaN,Albania,41.153300,20.168300,8/9/20,6411,199,3342,2870
53202,NaN,Algeria,28.033900,1.659600,8/9/20,35160,1302,24506,9352
53203,NaN,Andorra,42.506300,1.521800,8/9/20,955,52,839,64
53204,NaN,Angola,-11.202700,17.873900,8/9/20,1672,75,567,1030
...,...,...,...,...,...,...,...,...,...
53461,NaN,Sao Tome and Principe,0.186400,6.613100,8/9/20,878,15,800,63
53462,NaN,Yemen,15.552727,48.516388,8/9/20,1804,515,913,376
53463,NaN,Comoros,-11.645500,43.333300,8/9/20,399,7,369,23
53464,NaN,Tajikistan,38.861000,71.276100,8/9/20,7745,62,6484,1199


-----

# USA data

In [35]:
# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

try:
    os.remove('time_series_covid19_confirmed_US.csv')
    os.remove('time_series_covid19_deaths_US.csv')
except:
    pass

# download files
for url in urls:
    filename = wget.download(url)

100% [..........................................................................] 2208165 / 2208165

100% [..........................................................................] 1809285 / 1809285

In [36]:
us_confirmed = pd.read_csv('time_series_covid19_confirmed_US.csv')
us_deaths = pd.read_csv('time_series_covid19_deaths_US.csv')

In [37]:
us_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,225,226,231,247,247,247,253,257,267,280,280,280,280,301,303,307,309,310,310,310,312,312,313,314,314,314,314,319,319,330,337,337,337,337,349,351,354,356,356,356,368,375,389,397,411,411,411,418,418,449
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,30,30,30,30,30,30,30,30,30,31,31,31,31,31,31,31,31,31,33,33,33,36,36,37,37,37,37,38,38,38,38,38,40,40,40,40,42,42,42,44,45,46,46,46,47,47,48,48,49,49
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,43,43,43,43,43,47,48,50,50,50,50,50,52,58,59,59,60,61,63,63,66,66,66,66,67,68,72,72,72,72,72,73,74,77,77,77,77,78,82,85,85,86,89,89,90,92,92,94,94,95
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,30,30,32,32,33,34,34,34,35,36,36,37,37,48,48,48,50,50,52,54,57,58,60,60,66,67,70,71,75,78,82,84,89,91,92,95,96,97,99,101,104,108,109,110,111,117,123,135,140,144


In [38]:
us_deaths.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,...,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",55641,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",164229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",55144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",19483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",41959,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
id_vars = us_confirmed.columns[0:11]
us_dates = us_confirmed.columns[11:]
us_confirmed = us_confirmed.melt(id_vars=id_vars, var_name='Date', value_vars=us_dates, value_name='Confirmed')

id_vars = us_deaths.columns[0:12]
us_dates = us_deaths.columns[12:]
us_deaths = us_deaths.melt(id_vars=id_vars, var_name='Date', value_vars=us_dates,  value_name='Deaths')

In [40]:
us_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
678015,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",8/11/20,430
678016,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",8/11/20,115
678017,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",8/11/20,3240
678018,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",8/11/20,181


In [41]:
us_deaths

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",55641,1/22/20,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",164229,1/22/20,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",55144,1/22/20,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",19483,1/22/20,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",41959,1/22/20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678015,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",0,8/11/20,2
678016,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",0,8/11/20,1
678017,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",0,8/11/20,26
678018,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",0,8/11/20,0


In [42]:
us = pd.concat([us_deaths, us_confirmed[['Confirmed']]], axis=1)
us

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths,Confirmed
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",55641,1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",164229,1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",55144,1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",19483,1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",41959,1/22/20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678015,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,"Central Utah, Utah, US",0,8/11/20,2,430
678016,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,"Southeast Utah, Utah, US",0,8/11/20,1,115
678017,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,"Southwest Utah, Utah, US",0,8/11/20,26,3240
678018,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,"TriCounty, Utah, US",0,8/11/20,0,181


In [43]:
us.shape

(678020, 15)

In [44]:
try:
    os.remove('us_county_wise.csv')
except:
    pass

us.to_csv('us_county_wise.csv', encoding='utf_8', index=False)

In [45]:
yolo = us[us['Admin2']=='Yolo']

In [46]:
yolo.shape

(203, 15)

In [47]:
yolo.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths,Confirmed
661644,84006113,US,USA,840,6113.0,Yolo,California,US,38.682789,-121.901829,"Yolo, California, US",220500,8/7/20,42,1660
664984,84006113,US,USA,840,6113.0,Yolo,California,US,38.682789,-121.901829,"Yolo, California, US",220500,8/8/20,43,1690
668324,84006113,US,USA,840,6113.0,Yolo,California,US,38.682789,-121.901829,"Yolo, California, US",220500,8/9/20,43,1721
671664,84006113,US,USA,840,6113.0,Yolo,California,US,38.682789,-121.901829,"Yolo, California, US",220500,8/10/20,44,1834
675004,84006113,US,USA,840,6113.0,Yolo,California,US,38.682789,-121.901829,"Yolo, California, US",220500,8/11/20,44,1834


In [48]:
try:
    os.remove('yolo.csv')
except:
    pass

yolo.to_csv('yolo.csv', encoding='utf_8', index=False)

In [49]:
# china
# =====

china_province_wise = df[df['Country/Region']=='China']
china_province_wise['Province/State'].unique()

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin',
       'Tibet', 'Xinjiang', 'Yunnan', 'Zhejiang'], dtype=object)

In [50]:
# os.remove('china_province_wise.csv')
china_province_wise.to_csv('china_province_wise.csv', index=False)

In [51]:
china_province_wise.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
53809,Tianjin,China,39.3054,117.3230,8/11/20,205,3,202,0
53810,Tibet,China,31.6927,88.0924,8/11/20,1,0,1,0
53811,Xinjiang,China,41.1129,85.2401,8/11/20,875,3,337,535
53812,Yunnan,China,24.9740,101.4870,8/11/20,191,2,188,1
53813,Zhejiang,China,29.1832,120.0934,8/11/20,1275,1,1268,6


In [52]:
# Australia
# =========

australia_state_wise = df[df['Country/Region']=='Australia']
australia_state_wise['Province/State'].unique()
# os.remove('australia_state_wise.csv')
australia_state_wise.to_csv('australia_state_wise.csv', index=False)

In [53]:
# Canada
# ======

canada_state_wise = df[df['Country/Region']=='Canada']
canada_state_wise['Province/State'].unique()
# os.remove('canada_state_wise.csv')
canada_state_wise.to_csv('canada_state_wise.csv', index=False)

In [72]:
import country_converter as coco

cc = coco.CountryConverter()

def country_name_converter(df, col_name):
    some_names = list(df[col_name])
    standard_names = cc.convert(names = some_names, to = 'name_short')
    df['country2'] = standard_names
    df = df.replace('not found', np.nan)
    num_nulls = df['country2'].isnull().sum().sum()
    print('number of nulls:', num_nulls)
    df['country2'] = np.where(df['country2'].isna(), df[col_name], df['country2'])
    df = df.drop(col_name, axis=1)
    df = df.rename(columns={'country2': 'country'})
    return df

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53998 entries, 0 to 53997
Data columns (total 29 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Province/State                      53998 non-null  object 
 1   Country/Region                      53998 non-null  object 
 2   Lat                                 53998 non-null  float64
 3   Long                                53998 non-null  float64
 4   Date                                53998 non-null  object 
 5   Confirmed                           53998 non-null  int64  
 6   Deaths                              53998 non-null  int64  
 7   Recovered                           53998 non-null  int32  
 8   Active                              53998 non-null  int64  
 9   Percent                             34916 non-null  float64
 10  Region                              0 non-null      object 
 11  Population                          0 non